In [22]:
import pandas as pd

train_df = pd.read_csv("../data/train.csv")
urls = train_df["url"]
title_text = train_df["text"]

In [ ]:
from urllib import request
from bs4 import BeautifulSoup
import re
from tqdm import tqdm

# 전처리 할 특수 기호
pattern_punctuation = re.compile(r"[,!\"#&'()+\-/:<=>?@\[\]_`}{~]")
# 크롤링한 기사 제목
title_list = []

for url in tqdm(urls):
    try:
        html = request.urlopen(url)
        bsObject = BeautifulSoup(html, "html.parser")
        if bsObject.body.h2.span:
            title = bsObject.body.h2.span.string
            title = pattern_punctuation.sub('', title)
            title_list.append(title)
        else:
            title = bsObject.body.h4.string
            title = pattern_punctuation.sub('', title)
            title_list.append(title)
    except Exception as e:
        print(f"{e=}")
        print(f"{url=}")
        title_list.append("")

In [25]:
import os

result = pd.DataFrame(title_list)
result.columns=["text"]
os.makedirs("../preprocess", exist_ok=True)
result.to_csv("../preprocess/crawling.csv", index=False)

In [27]:
for index, row in result.iterrows():
    if row["text"] != "":
        train_df.loc[index, "text"] = row["text"]

train_df.to_csv("../preprocess/cleansing_train.csv", index=False)